In [2]:
import numpy as np
import random
import pandas as pd
from numpy import savetxt, loadtxt
import time
import math

class HeavyQueen9N8:
    def __init__(self, chess_dim=8, file_name = None):
        self.dim = chess_dim
        self.file_name = file_name
        self.chess_board = np.zeros((self.dim, self.dim)).astype(int)

    def init_board(self):
        if self.file_name == None:
            for i in range(self.dim):
                self.chess_board[random.randint(0, self.dim-1), i] = random.randint(1, 9)
            flag = 1
            while(flag):
                col = random.randint(0, self.dim-1)
                row = random.randint(0, self.dim-1)
                if self.chess_board[row, col] == 0:
                    self.chess_board[row, col] = random.randint(1,9)
                    flag = 0
            savetxt('test.txt', self.chess_board, delimiter=',')
            return self.chess_board
        else:
            self.__loadboard()
            return self.chess_board

    def __loadboard(self):
        load_mtx = loadtxt(self.file_name, delimiter=',').astype(int)
        self.n = len(load_mtx)
        self.chess_board = load_mtx
        print('loaded')
        return load_mtx

    def test(self):
        print(self.n)
        pass

In [1]:
class genetic_algo:
    def __init__(self, board, population = 100, crossover = 3, mutate = 0.1, time = 0, eli_num = 10, cul_num = 20, max_gen = 100):
        self.generationNum = 0 # indicate which generation is now, initial = 0
        self.population = list() # population in each generation
        self.pValue = population # num of population in each generation, setted by user, default 100 population/generation
        self.cost_list = list() # cost of each population
        self.crossover = crossover # in which postion to crossover, default colume 3
        self.mutate = mutate # the chance of mutation, default 10%
        self.initial_board = board # initial board, need to input
        self.init_col_list = list(np.nonzero(board.T)[0])
        self.init_row_list = list(np.nonzero(board.T)[1])
        self.weight_list = list() # weight of each queen
        self.dim = len(board) # get dim of board
        self.duration = time # set how much time to run the algorithm
        self.queen_num = len(self.init_col_list)
        self.prob_list = list() # probability of each population
        self.eli_num = eli_num # number of boards to be preserved in elitism, default 10
        self.cul_num = cul_num # number of boards to be deleted in culling, default 20
        self.next_population_list = list() # collectting generated population
        self.max_generation = max_gen # max num of generating, default 100
        self.max_time = time # max seconds of time to run, default 0 means not set
        self.min_cost_list = list() # a list of minimal cost in each generation
        self.min_pop_list = list() # a list of minimal board in each generation
        
    def attack_cost(self, row_list):  # calculate the cost of queens' attacks
        count = 0
        pos_row = row_list
        pos_col = self.init_col_list
        for i in range(len(pos_col)):
            row = pos_row[i]
            col = pos_col[i]
            # cal right
            for j in range(i+1, len(pos_col)):
                if pos_row[j] == row:
                    count += 1
            # cal below
            for j in range(i+1, len(pos_col)):
                if pos_col[j] == col:
                    count += 1
            # cal right-below
            for j in range(i+1, len(pos_col)):
                col_next = pos_col[j]
                row_next = pos_row[j]
                diff_col = col_next - col
                if row+diff_col < self.dim:
                    if row_next == row+diff_col:
                        count += 1
                else:
                    break
            # cal right_above
            for j in range(i+1, len(pos_col)):
                col_next = pos_col[j]
                row_next = pos_row[j]
                diff_col = col_next - col
                if row-diff_col > 0:
                    if row_next == row-diff_col:
                        count += 1
                else:
                    break
        return count * 100
    
    def init_weight_list(self): # initial the weight list
        for i in range(len(self.init_col_list)):
            self.weight_list.append(board[self.init_row_list[i]][self.init_col_list[i]])
    
    def move_cost(self, row_list): # calculate the move cost of current board with initial board
        cost = 0
        for i in range(len(self.init_col_list)):
            row = self.init_row_list[i]
            cur_row = row_list[i]
            if row != cur_row:
                cost = cost + math.pow(self.weight_list[i], 2) * abs(row - cur_row)
        return cost
    
    def rand_change_pos(self, i, row_list): # change the (i+1)th queen position without changing input
        row_list_new = row_list
        row = row_list_new[i]
        rand_row = random.randint(0, self.dim-1)
        while rand_row == row:
            rand_row = random.randint(0, self.dim-1)
        row_list_new[i] = rand_row
        return row_list_new

    def init_environment(self): # randomly change 1-2 queens in their column, and get the initial populations
        self.population.append(self.init_row_list)
        i = 1
        while i < self.pValue:
            # randomly select 1-2 colume to change queens' position
            num_q = random.randint(1,2)
            if num_q == 1:
                queen = random.randint(0, self.queen_num-1)
                row_list_new = self.rand_change_pos(queen, self.init_row_list[:])
            else:
                queen1 = random.randint(0, self.queen_num-1)
                queen2 = random.randint(0, self.queen_num-1)
                while queen1 == queen2:
                    queen2 = random.randint(0, self.queen_num-1)
                row_list_new = self.rand_change_pos(queen1, self.init_row_list[:])
                row_list_new = self.rand_change_pos(queen2, row_list_new)
            if row_list_new not in self.population:
                    i += 1
                    self.population.append(row_list_new)
    
    def cal_cost_list(self): # calculate the cost list using row list in populations
        if 0 != len(self.cost_list):
            self.cost_list = list()
        for i in self.population:
            cost = 0
            move_cost = self.move_cost(i)
            attack_cost = self.attack_cost(i)
            cost = move_cost + attack_cost
            self.cost_list.append(cost)
            
    def cal_prob(self, cost_list): # calculate the random selecting probability of each row list in population using its cost
        prob_list = list()
        temp = pd.DataFrame({'cost':cost_list})
        invert_cost_list = list(sum(temp['cost'])/temp['cost'])
        total_invert_cost = sum(invert_cost_list)
        for i in invert_cost_list:
            prob = i/total_invert_cost
            prob_list.append(prob)
        return prob_list
    
    def sort_cost_pop_list(self): # sort population list according cost list in ascending order
        data = pd.DataFrame({"c":self.cost_list, "p":self.population})
        data_sorted = data.sort_values(by=['c'])
        cost_sorted = list(data_sorted['c'])
        population_sorted = list(data_sorted['p'])
        return (cost_sorted, population_sorted)
    
    def elitism(self): # preserve best eli_num populations
        cost_sorted, population_sorted = self.sort_cost_pop_list()
        self.next_population_list.extend(population_sorted[:self.eli_num])
        
    def culling(self): # delete worst cul_num populations
        cost_sorted, population_sorted = self.sort_cost_pop_list()
        cul_population = population_sorted[:len(population_sorted) - self.cul_num]
        cul_cost = cost_sorted[:len(population_sorted) - self.cul_num]
        return (cul_cost, cul_population)
    
    def get_parent(self, prob_list, population_list): # randomly choose a population as parent
        prob = random.random()
        count = 0
        par = None
        for i in range(len(prob_list)):
            count += prob_list[i]
            if count >= prob:
                par = population_list[i]
                return par
    
    def cross_over(self, par1, par2): # give two populations to generate two new populations
        child1 = list()
        child2 = list()
        child1 = par1[:self.crossover] + par2[self.crossover:]
        child2 = par2[:self.crossover] + par1[self.crossover:]
        return (child1, child2)
        
        
    def mutation(self, child): # randomly mutation in child
        if self.mutate > 0:
            pro = random.random()
            if pro <= self.mutate:
                pos = random.randint(0, self.queen_num-1)
                child = self.rand_change_pos(pos, child)

    def processing(self):
        start_time = time.time()
        if 0 == len(self.population):
            print("=====================Initializing===========================")
            self.init_weight_list() # initial weight list
            self.init_environment() # initial populations
            self.cal_cost_list() # generate cost list for elistism and culling
        while self.generationNum < self.max_generation:
            self.elitism() # elitism
            cul_cost, cul_population = self.culling() # get cost and populations after culling
            self.min_cost_list.append(cul_cost[0])
            self.min_pop_list.append(cul_population[0])
            prob_list = self.cal_prob(cul_cost) # calculate probability of current populations
            while len(self.next_population_list) < self.pValue: # while population not reach setted num, generate child and put it in list
                par1 = self.get_parent(prob_list, cul_population)
                par2 = self.get_parent(prob_list, cul_population)
                child1, child2 = self.cross_over(par1, par2)
                self.mutation(child1)
                self.mutation(child2)
                self.next_population_list.append(child1)
                self.next_population_list.append(child2)
            self.population = self.next_population_list # update population into next generation
            self.next_population_list = list() # clean next population list
            self.cal_cost_list()
            self.generationNum += 1
            end_time = time.time()
            if self.max_time != 0: # checking if time of running setted
                if end_time - start_time >= self.max_time: # check if have reach setted running time
                    break
        
        cost, pop = self.sort_cost_pop_list()
        self.min_cost_list.append(cost[0])
        self.min_pop_list.append(pop[0])
        min_cost = self.min_cost_list[-1]
        min_pop = self.min_pop_list[-1]
        print('=====================Finishing===========================')
        self.clean()
        
        return (min_cost, min_pop, end_time-start_time, self.generationNum)
        
    def clean(self): # when finish, set augment to initial state, for user to run ag again.
        self.generationNum = 0
        self.population = list()
        self.cost_list = list()
        self.prob_list = list()
        self.min_cost_list = list()
        self.min_pop_list = list()
        
    def get_board(self, row_list): # form a numpy-array
        board = np.zeros((self.dim, self.dim)).astype(int)
        for i in range(len(self.init_col_list)):
            row = row_list[i]
            col = self.init_col_list[i]
            board[row][col] = self.weight_list[i]
        return board

In [3]:
test = HeavyQueen9N8(file_name = 'test.txt')
board = test.init_board()
print(board)

loaded
[[0 4 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 5 0 5]
 [4 0 0 6 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 8 0 1 0 5 0]]


In [30]:
GA_test = genetic_algo(board)

In [31]:
cost, population, times, gen_num = GA_test.processing()

=====================Initializing===========================
=====================Finishing===========================


In [32]:
population

[4, 0, 7, 3, 0, 6, 2, 7, 1]

In [33]:
cost

342.0

In [34]:
result = GA_test.get_board(population)
result

array([[0, 4, 0, 0, 2, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 5],
       [0, 0, 0, 0, 0, 5, 0, 0],
       [0, 0, 0, 6, 0, 0, 0, 0],
       [4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 8, 0, 0, 0, 5, 0]])

In [10]:
GA_test1 = genetic_algo(board, max_gen = 10000)

In [11]:
GA_test1.processing()

=====================Initializing===========================
=====================Finishing===========================


(342.0, [4, 0, 7, 3, 0, 6, 2, 7, 1], 54.891287088394165, 0)

In [8]:
GA_test2 = genetic_algo(board, time = 0.15) # it depends if we luck

In [9]:
GA_test2.processing()

=====================Initializing===========================
=====================Finishing===========================


(342.0, [4, 0, 7, 3, 0, 6, 2, 7, 1], 0.15361571311950684, 0)